In [ ]:
import math
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

import matplotlib.pyplot as plt

In [ ]:
# Load the data
df = pd.read_csv('all_matches.csv')


In [ ]:
df['season'].unique()

array(['2007/08', '2009', '2009/10', '2011', '2012', 2012, 2013, 2014,
       2015, 2016, 2017, 2019, 2018, '2018', '2019', '2020/21', '2021',
       2021, 2022, 2023, 2024], dtype=object)

In [ ]:
# Rename 'striker' column to 'Batsman' and 'Total_Runs' to 'total_runs'
df = df.rename(columns={'striker': 'Batsman', 'runs_off_bat': 'total_runs'})

In [ ]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'Batsman', 'non_striker', 'bowler',
       'total_runs', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [ ]:
# Filter data for Virat Kohli playing for Royal Challengers Bangalore
kohli_df = df[ (df['Batsman'] == 'V Kohli')]

In [ ]:
# Convert the 'start_date' to datetime format to extract year
kohli_df['start_date'] = pd.to_datetime(kohli_df['start_date'])
kohli_df['year'] = kohli_df['start_date'].dt.year


In [ ]:
kohli_df['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2019,
       2018, 2020, 2021, 2022, 2023, 2024], dtype=int32)

In [ ]:
kohli_df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'Batsman', 'non_striker', 'bowler',
       'total_runs', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year'],
      dtype='object')

In [ ]:
# Compute highest score by grouping by year and match_id, then taking the sum of total_runs within each match
kohli_df['match_id'] = kohli_df['match_id'].astype(str)  # Ensure match_id is a string for grouping
highest_scores = kohli_df.groupby(['year', 'match_id']).total_runs.sum().reset_index()
highest_scores = highest_scores.groupby('year').total_runs.max().reset_index()
highest_scores = highest_scores.rename(columns={'total_runs': 'highest_score'})

In [ ]:
# Calculate yearly statistics
yearly_stats = kohli_df.groupby('year').agg(
    total_runs=('total_runs', 'sum'),
    balls_faced=('ball', 'count'),
    sixes=('total_runs', lambda x: (x == 6).sum()),
    fours=('total_runs', lambda x: (x == 4).sum()),
    #fifties=('total_runs', lambda x: ((x >= 50) & (x < 100)).sum()),  # Number of fifties
    #hundreds=('total_runs', lambda x: (x >= 100).sum())
).reset_index()

In [ ]:
 kohli_df.groupby(['year','match_id'])['total_runs'].sum()



year  match_id
2008  335982       1
      335985      23
      335992      13
      335996      12
      335998       1
                  ..
2024  1426274     18
      1426279     51
      1426283     70
      1426290     42
      1426296     92
Name: total_runs, Length: 241, dtype: int64

In [ ]:
# Compute the total runs per match
match_runs = kohli_df.groupby(['year', 'match_id'])['total_runs'].sum().reset_index()

# Rename the column to 'RUNS'
match_runs = match_runs.rename(columns={'total_runs': 'RUNS'})

In [ ]:
match_runs.head()

,year,match_id,RUNS
0,2008,335982,1
1,2008,335985,23
2,2008,335992,13
3,2008,335996,12
4,2008,335998,1


In [ ]:
# Calculate the number of 50s and 100s per year
fifties = match_runs.groupby('year')['RUNS'].apply(lambda x: (x.between(50, 99)).sum()).reset_index()
fifties = fifties.rename(columns={'RUNS': 'fifties'})

centuries = match_runs.groupby('year')['RUNS'].apply(lambda x: (x >= 100).sum()).reset_index()
centuries = centuries.rename(columns={'RUNS': 'centuries'})



In [ ]:
# Merge the number of 50s and 100s with yearly_stats
yearly_stats = yearly_stats.merge(fifties, on='year').merge(centuries, on='year')

In [ ]:

# Merge highest scores with yearly_stats
yearly_stats = yearly_stats.merge(highest_scores, on='year')

In [ ]:
#Calculate strike rate and round to 2 decimal places

yearly_stats['strike_rate'] = round((yearly_stats['total_runs'] / yearly_stats['balls_faced']) * 100, 2)

# Save the processed data to a xlsx file
yearly_stats.to_excel('/content/virat_kohli_yearly_stats_FH.xlsx', index=False)

# Display the dataframe
yearly_stats

,year,total_runs,balls_faced,sixes,fours,fifties,centuries,highest_score,strike_rate
0,2008,165,168,4,18,0,0,38,98.21
1,2009,246,225,8,22,1,0,50,109.33
2,2010,307,216,12,26,1,0,58,142.13
3,2011,557,473,16,55,4,0,71,117.76
4,2012,364,333,10,33,2,0,73,109.31
5,2013,639,471,22,65,7,0,99,135.67
6,2014,359,301,16,23,2,0,73,119.27
7,2015,505,395,23,35,3,0,82,127.85
8,2016,973,655,38,84,7,4,113,148.55
9,2017,308,257,11,23,4,0,64,119.84


In [ ]:
# Group by year and opposing team, then sum the runs
kohli_runs_by_apposition= kohli_df.groupby(['year', 'bowling_team'])['total_runs'].sum().reset_index()

In [ ]:
kohli_runs_by_apposition

,year,bowling_team,total_runs
0,2008,Chennai Super Kings,22
1,2008,Deccan Chargers,47
2,2008,Delhi Daredevils,1
3,2008,Kings XI Punjab,55
4,2008,Kolkata Knight Riders,1
...,...,...,...
122,2024,Lucknow Super Giants,22
123,2024,Mumbai Indians,3
124,2024,Punjab Kings,169
125,2024,Rajasthan Royals,113


In [ ]:
# Save the processed data to a xlsx file
kohli_runs_by_apposition.to_excel('/content/kohli_runs_by_apposition.xlsx', index=False)

In [ ]:
kohli_runs_by_apposition['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023, 2024], dtype=int32)